------------------------------------------
<a id='top'></a>
## Session contents
### [10. Working with Timestamps](#timestamps)
### [11. Resampling data](#resampling_data)
### [12.  Merging by time](#merge_time)
### [Exercise set 4](#exercises4)

-------------------
<a id='timestamps'></a>
## 10. Working with Timestamps

Since financial data are often time series, it makes sense to set the index of our DataFrame to be the timestamp of each event. Pandas provides a __pd.to_datetime()__ function to convert strings or other date objects into datetime64 objects that pandas likes to work with.

When a column of timestamps (datetime64 objects) is set as the index, a lot of time-series methods for the DataFrame become available.

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

http://pandas.pydata.org/pandas-docs/stable/timeseries.html
http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.11-Working-with-Time-Series.ipynb#Dates-and-times-in-pandas:-best-of-both-worlds

http://pandas.pydata.org/pandas-docs/version/0.19.0/generated/pandas.DataFrame.asof.html


<div class="alert alert-success">

If you have just started here or would like to refresh your df_qte object, run the line below.

In [26]:
import pandas as pd
import numpy as np
import datetime as dt

In [27]:
df_qte = pd.read_csv('data/quote.csv')
df_qte.columns = ['Index', 'Time', 'BidPrice', 'BidSize', 'AskPrice', 'AskSize']
df_qte.drop(columns='Index', inplace=True)

In [28]:
df_qte.head()

,Time,BidPrice,BidSize,AskPrice,AskSize
0,2014-01-02 00:45:01.020274,0.0,3.0,233.98,1.0
1,2014-01-02 00:45:01.520217,0.0,4.0,0.00,3.0
2,2014-01-02 00:45:02.020301,0.0,4.0,0.00,4.0
3,2014-01-02 01:08:22.692413,0.0,4.0,0.00,3.0
4,2014-01-02 01:09:24.700202,0.0,1.0,0.00,3.0


We would like to replace the index of our DataFrame with the values in the column 'Time', converting these values into datetime objects in the process. 

Check for yourself what type of an object the values in column 'Time' are currently stored as.

In [30]:
df_qte.loc[df_qte.index[0], 'Time']

'2014-01-02 00:45:01.020274'

In [31]:
type(df_qte.Time[0])

str

Were we to set the Time column as the index right now, pandas would not recognise it as an index of timestamps.

Let's try out the pd.to_datetime function on the first Time value.

In [33]:
import datetime as dt

In [32]:
pd.to_datetime(df_qte.Time[0])

Timestamp('2014-01-02 00:45:01.020274')

This function works for a variety of time objects and formats.

In [3]:
pd.to_datetime(dt.datetime(2017, 5, 5, 12, 30))

Timestamp('2017-05-05 12:30:00')

In [2]:
import pandas as pd
import datetime as dt

In [ ]:
pd.to_datetime(dt)

In [43]:
pd.to_datetime(265, unit='D')

Timestamp('1970-09-23 00:00:00')

The latter format is called Unix time or Unix epoch and is the standard high-precision timestamp format in our trading systems. It is defined as the number of seconds (or milli/micro/nanoseconds) since midnight 1 January 1970 (1/1/1970 00:00:00 GMT).

We can set Time as a DateTimeIndex by first converting the column's values to pandas Timestamp objects, then setting it as the index in the usual way.

In [44]:
df_qte['Time'] = pd.to_datetime(df_qte['Time'])
df_qte = df_qte.set_index('Time')

In [45]:
df_qte.head()

,BidPrice,BidSize,AskPrice,AskSize
Time,,,,
2014-01-02 00:45:01.020274,0.0,3.0,233.98,1.0
2014-01-02 00:45:01.520217,0.0,4.0,0.00,3.0
2014-01-02 00:45:02.020301,0.0,4.0,0.00,4.0
2014-01-02 01:08:22.692413,0.0,4.0,0.00,3.0
2014-01-02 01:09:24.700202,0.0,1.0,0.00,3.0


Check that the index of the DataFrame is now a DatetimeIndex.

In [46]:
df_qte.index

DatetimeIndex(['2014-01-02 00:45:01.020274', '2014-01-02 00:45:01.520217',
               '2014-01-02 00:45:02.020301', '2014-01-02 01:08:22.692413',
               '2014-01-02 01:09:24.700202', '2014-01-02 01:09:25.200292',
               '2014-01-02 01:10:12.206330', '2014-01-02 01:10:57.712263',
               '2014-01-02 01:15:00.241778', '2014-01-02 01:15:00.742388',
               ...
               '2014-01-02 08:14:55.323523', '2014-01-02 08:14:55.823454',
               '2014-01-02 08:14:56.323602', '2014-01-02 08:14:57.323818',
               '2014-01-02 08:14:57.823685', '2014-01-02 08:14:58.323779',
               '2014-01-02 08:14:58.824068', '2014-01-02 08:14:59.824308',
               '2014-01-02 08:15:00.324603', '2014-01-02 08:17:00.338057'],
              dtype='datetime64[ns]', name=u'Time', length=25136, freq=None)

The DateTimeIndex is a lot more flexible than a regular index in how data can be selected. We can use loc with a datetime object to return a row at that exact time.

In [ ]:
df_qte.loc[dt.datetime(2014, 1, 2, 1, 8, 22, 692413)]

We can also use loc with a datetime string to return all rows that match the specified datetime, up to the level of detail provided.

In [ ]:
df_qte.loc['2014-01-02 00:45']

Slicing with datetime strings allows for selection of a specified time period.

In [ ]:
df_qte.loc['2014-01-02 00:00':'2014-01-02 01:14']

To select the same time period across multiple days, we can use the between_time method. This can be useful for separating out morning and afternoon trading sessions, or filtering out the auction period.

In [ ]:
df_qte.between_time('00:00', '01:15')

Finally, the df.asof() method takes a timestamp and returns the most recent non-NaN row. Try finding the last quote prices as at 01:15 with this method.

<div class="alert alert-block alert-warning">
### Working with timezones - OneTick

Depending on the timezone parameter selected in your otq query, your timestamps may be imported as a tz-aware object. This is an object that belongs to the datetime (dt) package. 

If you would like to convert a tz-aware index into a tz-naïve index, see the following Stackoverflow post. This is especially relevant when you are pulling data from different databases in OneTick.

http://stackoverflow.com/questions/16628819/convert-pandas-timezone-aware-datetimeindex-to-naive-timestamp-but-in-certain-t


[Back to top](#top)

------------
<a id='resampling_data'></a>
## 11. Resampling data

The main method used to resample data is __df.resample__, which is available when the DataFrame/Series object has a valid DatetimeIndex.

It is common to combine resampling with __dropna()__ or __fillna()__ methods as resampling upwards (higher frequency than the original data) will create null values.

<div class="alert alert-block alert-info">

<span style="color:green">YouTube video</span>

Watch the following video until around the 2 hour 13 minute mark.

https://www.youtube.com/watch?v=JNfxr4BQrLk&start=6956

**Key methods covered:**

    df.resample - similar to .asfreq
    df.fillna - fills NaNs according to specified logic

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.11-Working-with-Time-Series.ipynb#Resampling,-Shifting,-and-Windowing

<div class="alert alert-success">

If you have just started here or would like to refresh your df_qte object, run the line below.

In [7]:
import pandas as pd
import numpy as np
import datetime as dt

In [47]:
df_qte = pd.read_csv('data/quote.csv')
df_qte.columns = ['Index', 'Time', 'BidPrice', 'BidSize', 'AskPrice', 'AskSize']
df_qte.drop(columns='Index', inplace=True)
df_qte['Time'] = pd.to_datetime(df_qte['Time'])

Let's try resampling the data into 5 minute buckets using the df.resample() method.

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html

If we simply call .resample with a frequency (e.g. '100ms', '1s', '5min', '2h', '1D'), we will end up with a DatetimeIndexResampler object.

In [48]:
df_qte = df_qte.set_index('Time')

This is because we have yet to specify how we want to resample the data. Try applying a __.last(), .mean()__, or __.sum()__ operation after .resample. In what situations would each be the appropriate operation to use?

In [50]:
df_qte.resample('20min').mean()

,BidPrice,BidSize,AskPrice,AskSize
Time,,,,
2014-01-02 00:40:00,0.000000,3.666667,77.993333,2.666667
2014-01-02 01:00:00,232.095637,4.119691,232.609479,2.976834
2014-01-02 01:20:00,233.880870,3.797419,233.937497,3.739057
2014-01-02 01:40:00,233.113115,3.017478,233.155568,3.274657
2014-01-02 02:00:00,232.773070,3.443908,232.813528,3.567551
2014-01-02 02:20:00,232.617436,3.517094,232.658084,3.338319
2014-01-02 02:40:00,231.648899,3.708064,231.690129,4.856386
2014-01-02 03:00:00,231.689362,3.382567,231.730646,3.464084
2014-01-02 03:20:00,231.518236,3.401211,231.560810,3.281605


Resampling often results in NaNs. This happens when no data comes through within that sample period. 

It is appropriate to forward fill the NaNs if the data represents updates to the state of some object (e.g. an order book, a volatility curve, an autotrader's parameters). If the data represents individual events like trade ticks, then forward filling will overcount trade volumes. In this case, it is more appropriate to fill NaNs with 0, to drop the NaNs, or to simply keep them in the DataFrame.

Sometimes our data consists of distinct groups and we want to apply a resampling operation - for instance, resample all relative expiries of a volatility surface. Let's load up that data again.

In [ ]:
df_vols = pd.read_csv('data/vols.csv')
df_vols.TIMESTAMP = pd.to_datetime(df_vols.TIMESTAMP)
df_vols = df_vols.set_index('TIMESTAMP', drop=True)
df_vols.head()

Try resampling and getting the first update every 10 minutes.

This operation isn't what we wanted, since we get a relative expiry at random for each time bucket. We can instead perform a resample after a __groupby__ before getting the first entry, so that we resample each relative expiry properly.

Try getting the volatility values for each relative expiry every 10 mins by:
1. Grouping by RELATIVE_EXPIRY
2. Resampling
3. Selecting only the VOLATILITY column
4. Unstacking RELATIVE_EXPIRY from the index to the columns.
5. Forward-filling NaNs, if any.

In [ ]:
#solutions
%load resample_sol1.py

[Back to top](#top)

------------
<a id='merge_time'></a>
# 12. Merging by time

The joins we covered in the previous session were __exact__ joins. That is, the data were only joined together if the values of each key were identical.

If we want to join two time series together, we'll find that the timestamps rarely match exactly. The join we are after is usually along the lines of "for each row in [left source], give me the most recent data from [right source]".

The classic example in trading data is joining trade and quote data together. We want to know, for each trade, what the state of the order book was at that time.

Pandas implements this kind of join with the function

    pd.merge_asof()
    
which is very similar to pd.merge. There a few additional arguments that are specific to pd.merge_asof:

    direction - whether to find the matching row from the right source, either 'backward' (default), 'forward' (i.e. next row), or 'nearest'
    tolerance - only match if the difference between indexes is below this number, e.g. tolerance=pd.Timedelta('1s') will only join the right source if its time was within 1 second of the left source
    by - do the time join for each value in these columns separately. E.g. by='FEEDCODE' will perform the join for each feedcode separately. 

<div class="alert alert-block alert-info">

<span style="color:green">Additional resources</span>

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.merge_asof.html

Let's load up both quote and trade data.

In [ ]:
df_qte = pd.read_csv('data/quote.csv')
df_qte.columns = ['Index', 'Time', 'BidPrice', 'BidSize', 'AskPrice', 'AskSize']
df_qte.Time = pd.to_datetime(df_qte.Time)
df_qte.drop(columns='Index', inplace=True)
df_qte = df_qte.set_index('Time', drop=True)
df_qte.head()

In [ ]:
df_trd = pd.read_csv('data/trade.csv')
df_trd.TIMESTAMP = pd.to_datetime(df_trd.TIMESTAMP)
df_trd = df_trd.set_index('TIMESTAMP', drop=True)
df_trd.head()

For each trade, we can join on the most recent quote with pd.merge. Since our join key, the timestamps, are in each DataFrame's index, we have to set left_index=True and right_index=True.

In [ ]:
df = pd.merge_asof(df_trd, df_qte, left_index=True, right_index=True)
df

It looks like our trade prices don't always match up with the best bid or offer, i.e. there are synchronisation issues. Try to calculate the percentage of trades that don't line up with our quote data, using conditional expressions.

In [ ]:
#solutions
%load time_sol1.py

[Back to top](#top)

------------
<a id='exercises4'></a>
# Exercise set 4

Calculate the 30-minute price change for each trade, and plot the price changes as a time series and a histogram. Your 30-minute delayed price should be the midpoint of the nearest quote update. (Hint: Use __pd.merge_asof__, __df.tshift__, and __df.plot.hist__.)